In [1]:
import pandas as pd
import torch

diabetes_df = pd.read_csv("../WEEK16/diabetes.csv")
diabetes_df.head()

,Pregnancies,Glucose,BloodPressure,SkinThickness,Insulin,BMI,DiabetesPedigreeFunction,Age,Outcome
0,6,148,72,35,0,33.6,0.627,50,1
1,1,85,66,29,0,26.6,0.351,31,0
2,8,183,64,0,0,23.3,0.672,32,1
3,1,89,66,23,94,28.1,0.167,21,0
4,0,137,40,35,168,43.1,2.288,33,1


In [2]:
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler

X = diabetes_df.drop('Outcome', axis=1).values
y = diabetes_df['Outcome'].values

# Split into training and test set
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.3, random_state=42, stratify=y)

# #Standardize
sc= StandardScaler()
X_train=sc.fit_transform(X_train)
X_test=sc.fit_transform(X_test)
# print(X_train)
# print(y_train)

In [3]:
import torch.nn as nn
import torch.nn.functional as F #where the activation functions are

#create tensors = matrices 
X_train = torch.FloatTensor(X_train) 
X_test = torch.FloatTensor(X_test)

y_train = torch.LongTensor(y_train)
y_test = torch.LongTensor(y_test)

print(X_train)

tensor([[ 0.9314,  2.0179,  0.7807,  ...,  0.4315, -0.3748,  0.6321],
        [ 0.6326, -1.1486,  0.4654,  ..., -0.1198, -0.2942,  0.7170],
        [-0.5625, -0.4769, -0.2703,  ..., -0.2096,  2.7452,  0.0381],
        ...,
        [-0.8613, -0.7648,  0.0450,  ...,  0.7648, -0.7838, -0.3014],
        [ 0.6326,  2.2099,  1.2010,  ...,  0.4315, -0.6047,  2.7537],
        [ 0.0351,  0.7385, -0.5856,  ..., -0.3378, -0.5778,  0.2927]])


In [4]:
#artificial neural network
class ANN_Model(nn.Module):
    def __init__(self, input_features=8,hidden1=20,hidden2=20,out_features=2):
        super().__init__() #super is a computed indirect reference. So, it isolates changes
        # and makes sure that children in the layers of multiple inheritence are calling
        #the right parents
        self.layer_1_connection = nn.Linear(input_features, hidden1)
        self.layer_2_connection = nn.Linear(hidden1, hidden2)
        self.out = nn.Linear(hidden2, out_features)
        
    def forward(self, x):
        #apply activation functions
        x = F.relu(self.layer_1_connection(x))
        x = F.relu(self.layer_2_connection(x))
        x = self.out(x)
        return x

In [5]:
torch.manual_seed(42)

#create instance of model
ann = ANN_Model()

In [6]:
#loss function
loss_function = nn.CrossEntropyLoss()

#optimizer
optimizer = torch.optim.Adam(ann.parameters(),lr=0.01)

In [7]:
#run model through multiple epochs/iterations
final_loss = []
n_epochs = 500
for epoch in range(n_epochs):
    y_pred = ann.forward(X_train)
    loss = loss_function(y_pred, y_train)
    final_loss.append(loss)
    
    if epoch % 10 == 1:
        print(f'Epoch number: {epoch} with loss: {loss}')
        
    optimizer.zero_grad() #zero the gradient before running backwards propagation
    loss.backward() 
    optimizer.step() #perform one optimization step each epoch

Epoch number: 1 with loss: 0.6474701762199402
Epoch number: 11 with loss: 0.5270779132843018
Epoch number: 21 with loss: 0.45391377806663513
Epoch number: 31 with loss: 0.4234801232814789
Epoch number: 41 with loss: 0.39819812774658203
Epoch number: 51 with loss: 0.3721073269844055
Epoch number: 61 with loss: 0.3437724709510803
Epoch number: 71 with loss: 0.31378453969955444
Epoch number: 81 with loss: 0.28582650423049927
Epoch number: 91 with loss: 0.25994443893432617
Epoch number: 101 with loss: 0.23771321773529053
Epoch number: 111 with loss: 0.21422426402568817
Epoch number: 121 with loss: 0.19071198999881744
Epoch number: 131 with loss: 0.17592614889144897
Epoch number: 141 with loss: 0.15840473771095276
Epoch number: 151 with loss: 0.14416688680648804
Epoch number: 161 with loss: 0.1284773200750351
Epoch number: 171 with loss: 0.11511360853910446
Epoch number: 181 with loss: 0.1032460555434227
Epoch number: 191 with loss: 0.09050065279006958
Epoch number: 201 with loss: 0.0789993

In [8]:
#predictions
y_pred = []

with torch.no_grad(): #decreases memory consumption
    for i, data in enumerate(X_test):
        prediction = ann(data)
        y_pred.append(prediction.argmax()) #returns index with max element in each prediction set

In [9]:
from sklearn.metrics import classification_report

print(classification_report(y_test, y_pred))

              precision    recall  f1-score   support

           0       0.75      0.81      0.78       150
           1       0.59      0.51      0.54        81

    accuracy                           0.70       231
   macro avg       0.67      0.66      0.66       231
weighted avg       0.69      0.70      0.70       231



In [10]:
with torch.no_grad(): #decreases memory consumption
    for i, data in enumerate(X_test):
        print(ann(data))

tensor([ 5.0319, -5.4252])
tensor([-2.6168,  1.3062])
tensor([ 2.7535, -4.9109])
tensor([ 0.2514, -1.6017])
tensor([ 10.2738, -12.3366])
tensor([ 11.4650, -14.2435])
tensor([-1.3114, -3.3055])
tensor([-9.7475,  7.0411])
tensor([ 26.6600, -26.6981])
tensor([-0.1213, -2.8400])
tensor([ 15.4304, -15.7240])
tensor([-9.3898, -1.8246])
tensor([  9.3752, -10.7215])
tensor([ 2.9755, -4.7473])
tensor([-4.2856,  2.1033])
tensor([-11.7091,   8.8094])
tensor([-3.9440, -0.6266])
tensor([ 7.1054, -8.0366])
tensor([  8.7246, -10.4487])
tensor([ 6.7393, -8.1200])
tensor([-4.5387,  2.7633])
tensor([-0.0896, -3.5822])
tensor([-1.5577, -2.6846])
tensor([ 22.5015, -22.7031])
tensor([-9.7996,  7.3229])
tensor([ 5.1007, -6.6488])
tensor([ 2.5190, -5.1220])
tensor([ 2.2193, -2.2844])
tensor([ 1.6401, -1.7862])
tensor([-5.5820,  1.5395])
tensor([-12.2231,   6.9077])
tensor([ 5.1005, -5.9326])
tensor([-1.6240, -0.7265])
tensor([  8.5741, -12.6062])
tensor([ 38.3464, -35.0321])
tensor([-7.6075,  5.0630])
tensor

In [11]:
y_pred

[tensor(0),
 tensor(1),
 tensor(0),
 tensor(0),
 tensor(0),
 tensor(0),
 tensor(0),
 tensor(1),
 tensor(0),
 tensor(0),
 tensor(0),
 tensor(1),
 tensor(0),
 tensor(0),
 tensor(1),
 tensor(1),
 tensor(1),
 tensor(0),
 tensor(0),
 tensor(0),
 tensor(1),
 tensor(0),
 tensor(0),
 tensor(0),
 tensor(1),
 tensor(0),
 tensor(0),
 tensor(0),
 tensor(0),
 tensor(1),
 tensor(1),
 tensor(0),
 tensor(1),
 tensor(0),
 tensor(0),
 tensor(1),
 tensor(0),
 tensor(0),
 tensor(1),
 tensor(0),
 tensor(0),
 tensor(0),
 tensor(0),
 tensor(0),
 tensor(0),
 tensor(0),
 tensor(0),
 tensor(1),
 tensor(0),
 tensor(0),
 tensor(0),
 tensor(0),
 tensor(1),
 tensor(0),
 tensor(1),
 tensor(0),
 tensor(0),
 tensor(1),
 tensor(0),
 tensor(0),
 tensor(0),
 tensor(0),
 tensor(0),
 tensor(0),
 tensor(0),
 tensor(0),
 tensor(0),
 tensor(0),
 tensor(0),
 tensor(0),
 tensor(0),
 tensor(1),
 tensor(0),
 tensor(0),
 tensor(0),
 tensor(0),
 tensor(0),
 tensor(1),
 tensor(0),
 tensor(0),
 tensor(0),
 tensor(0),
 tensor(0),
 ten

In [12]:
if epoch == 500 and loss < 0.1:
            #predictions 
            y_pred = []
            y_pred_prob = []

            with torch.no_grad():
                for i, data in enumerate(X_test):
                    prediction = ann(data)
                    y_pred.append(prediction.argmax())
                    #prob = F.softmax(ann(data), dim=1)
                    #top_p, top_class = prob.topk(1, dim = 1)
                    #y_pred_prob.append(top_p)

tensor([[-0.2220,  0.2730,  0.4480,  ..., -0.4052, -0.4010,  0.1271],
        [ 0.0710, -0.3540, -0.2502,  ...,  0.3897,  1.4114, -0.5597],
        [ 2.1222,  0.6611,  1.2460,  ...,  0.6133, -0.6158,  1.5866],
        ...,
        [ 2.7082, -0.4436,  0.1488,  ...,  0.6133, -0.8525,  1.0715],
        [-0.5150,  1.1089,  1.0465,  ..., -0.0077,  1.1000,  2.8743],
        [-0.8080, -0.1749,  0.0490,  ...,  0.3649,  0.2405, -0.0446]])